# 🥋 Lekcja 12: Retain Graph (Życie po śmierci grafu)

Domyślny cykl życia w PyTorch:
1.  **Forward:** Budujemy graf, zapisujemy tensory pośrednie.
2.  **Backward:** Używamy grafu do policzenia gradientów.
3.  **Destrukcja:** Graf jest usuwany z pamięci (Free Memory).

Jeśli spróbujesz zrobić `.backward()` drugi raz na tym samym wyjściu, dostaniesz błąd, bo "mapa drogowa" już nie istnieje.

**Kiedy potrzebujemy `retain_graph=True`?**
1.  **Multi-Task Learning:** Masz jedną sieć, ale dwie różne funkcje straty (Loss A i Loss B), które chcesz aplikować sekwencyjnie.
2.  **Wizualizacja:** Chcesz podejrzeć gradienty przed wykonaniem "prawdziwego" kroku.
3.  **GANy:** Czasami przy skomplikowanych pętlach treningowych Dyskryminatora i Generatora.

Zasymulujemy ten błąd i go naprawimy.

In [1]:
import torch
import torch.nn as nn

# Prosta sieć
x = torch.randn(1, 10)
w = torch.randn(10, 1, requires_grad=True)

# Forward pass
y = x @ w
loss = y.sum()

print("Graf zbudowany.")
print(f"Loss fn: {loss.grad_fn}")

# Pierwszy Backward - Standardowy
loss.backward()
print("Pierwszy backward: SUKCES")

# Drugi Backward - Na tym samym grafie
try:
    loss.backward()
except RuntimeError as e:
    print("\n🚫 BŁĄD (Zgodnie z planem):")
    print(e)

Graf zbudowany.
Loss fn: <SumBackward0 object at 0x000001F0FBFB0520>
Pierwszy backward: SUKCES

🚫 BŁĄD (Zgodnie z planem):
Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.


## Scenariusz: Dwie niezależne straty

Wyobraź sobie, że trenujesz model, który ma:
1.  Dobrze klasyfikować obrazki (`Loss_Main`).
2.  Mieć małe wagi (Regularyzacja L2 - `Loss_Reg`).

Chcesz policzyć wpływ obu tych strat oddzielnie (np. żeby zalogować gradienty dla każdej z nich osobno przed zsumowaniem).

In [2]:
# Resetujemy wagi i gradienty
w = torch.randn(10, 1, requires_grad=True)
x = torch.randn(1, 10)

# 1. Forward
y = x @ w

# 2. Definiujemy dwie różne straty na podstawie tego samego 'y'
loss_main = (y - 1).pow(2).sum()  # Chcemy, żeby wynik był 1
loss_reg  = w.pow(2).sum()        # Chcemy, żeby wagi były małe

print("Mamy dwie straty wiszące na jednym grafie.")

# 3. Backward dla pierwszej straty
# WAŻNE: retain_graph=True
# Mówimy: "Policz gradienty dla loss_main, ale NIE NISZCZ grafu (x@w), bo loss_reg też go potrzebuje!"
loss_main.backward(retain_graph=True)

print(f"Gradient po Loss Main: {w.grad.view(-1)[:3]}... (część)")

# 4. Backward dla drugiej straty
# Teraz możemy zniszczyć graf (domyślnie retain_graph=False)
loss_reg.backward()

# Gradienty się ZSUMOWAŁY (Accumulation)
print(f"Gradient po obu stratach: {w.grad.view(-1)[:3]}... (suma)")

Mamy dwie straty wiszące na jednym grafie.
Gradient po Loss Main: tensor([-2.3429, -5.5241, -4.7889])... (część)
Gradient po obu stratach: tensor([-1.1643, -7.6936, -4.6166])... (suma)


## `retain_graph` vs `create_graph`

To częste nieporozumienie na rekrutacjach.

1.  **`retain_graph=True`**:
    *   "Nie kasuj buforów pośrednich po backwardzie".
    *   Potrzebne, gdy robisz **wiele backwardów na tym samym forwardzie**.

2.  **`create_graph=True`**:
    *   "Traktuj proces liczenia gradientu jako operację, którą też można różniczkować".
    *   Buduje graf pochodnej.
    *   Potrzebne do **pochodnych wyższego rzędu** (Hessian, MAML - notatnik 11 i 75).

**Przykład:** Czy `retain_graph` zużywa dużo pamięci?
Tak! Trzyma w VRAM całą historię aktywacji. Dlatego używaj tego tylko wtedy, gdy musisz. W 99% przypadków lepiej zsumować straty (`total_loss = loss1 + loss2`) i zrobić jeden `backward()`.

In [3]:
# TEST PAMIĘCI (Zrozumienie ryzyka)

# Duży tensor
huge = torch.randn(1000, 1000, requires_grad=True)
y = huge * 2
loss = y.sum()

# Backward z zatrzymaniem grafu
loss.backward(retain_graph=True)

# Tutaj graf (i tensor 'huge' w pamięci grafu) NADAL WISI w RAM.
# Dopiero gdy zrobimy kolejny backward bez retain, albo usuniemy zmienną, pamięć zostanie zwolniona.

loss.backward(retain_graph=False) 
# Teraz graf posprzątany.
print("Pamięć zwolniona.")

Pamięć zwolniona.


## 🥋 Black Belt Summary

1.  Domyślnie PyTorch jest **agresywny w sprzątaniu**. Po `.backward()` bufory znikają.
2.  Używaj `retain_graph=True` **TYLKO** wtedy, gdy musisz wywołać `.backward()` wielokrotnie na tym samym pod-grafie.
3.  **Alternatywa:** Zamiast robić dwa backwardy:
    ```python
    loss1.backward(retain_graph=True)
    loss2.backward()
    ```
    Zazwyczaj lepiej (szybciej i lżej dla pamięci) jest zrobić:
    ```python
    total_loss = loss1 + loss2
    total_loss.backward()
    ```
    Wtedy PyTorch sam ogarnie graf raz, a porządnie.